In [3]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [4]:
import numpy as np
from numpy.typing import ArrayLike
import json
from traitlets import observe
from pathlib import Path
data = json.loads(Path("/Users/mpiekenbrock/Downloads/miserables.json").read_text())

node_ids = np.unique([n["id"] for n in data["nodes"]])
src_ids = np.array([e["source"] for e in data["links"]])
tgt_ids = np.array([e["target"] for e in data["links"]])

src_ids = np.searchsorted(node_ids, src_ids)
tgt_ids = np.searchsorted(node_ids, tgt_ids)
node_ids = np.searchsorted(node_ids, node_ids)

edgelist = np.c_[src_ids, tgt_ids]

In [10]:
from pixiplex import Pixinet
p = Pixinet(node_ids, edgelist, 400, 400) # indeed, init is called after the Python initialization

In [11]:
p

Pixinet(height=400, node_ids=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22…

In [24]:
# widget.node_color = ["0xff0000"]*len(widget.node_ids)
widget.node_color = ["0xff0000" if i % 2 == 0 else "0x00ff00" for i in widget.node_ids]

In [59]:
# widget.send({ "type" : "msg:sync_node_coordinates"})
# widget._x

In [60]:
widget.send({ "type" : "msg:init_force"})

In [61]:
widget.send({ "type" : "msg:center_graph"})

In [5]:
widget.send({ "type" : "msg:custom", "foo":"bar" })

In [ ]:
# %% Embed statically -- this works!
# import ipywidgets.embed
# ipywidgets.embed.embed_minimal_html("forcegraph_embed.html", views=[widget], drop_defaults=False)